# WeatherMind: AI-Powered Weather & Fashion Assistant

## Overview
This notebook demonstrates a simple AI assistant that can:
- Provide real-time weather information for major cities
- Suggest fashion trends for different locations
- Perform basic mathematical calculations
- Remember conversation context

# Table of Contents

1. Setup	                        Basic imports and API configuration
2. Simple AI Chat	                Basic Gemini query function and test
3. Enhanced Bot	                    Calculator + chatbot with routing and test
4. Weather & Fashion	            Real weather/fashion data integration and test
5. Memory System	               	Full assistant with conversation memory and test
6. Final Demo	                 	Custom query demonstration


# 1. Setup and Dependencies {#setup}

Installing required packages and importing necessary libraries for our weather and fashion assistant.


In [58]:
!pip install -q google-generativeai


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.4 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [59]:
pip install google-generativeai langgraph langchain langchain-community


Note: you may need to restart the kernel to use updated packages.


# Configuration 

Setting up API keys and global variables for our assistant.



In [ ]:
import google.generativeai as genai

GOOGLE_API_KEY = "********"
genai.configure(api_key=GOOGLE_API_KEY)


In [85]:
import time
import google.generativeai as genai

def gemini_flash_qury(promt):
    modl = genai.GenerativeModel('gemini-1.5-flash')
    respons = modl.generate_content(promt)
    
    print("respons:")
    for chnk in respons:
        print(chnk.text)
        time.sleep(0.1)
    
    return respons.text


cheacking the api works properly or not 

In [86]:

response = gemini_flash_query("Explain blockchain technology using a pizza delivery analogy. Keep it to 3 paragraphs maximum.")
print(response)

Response:
Imagine you order a pizza.  A blockchain is like a digital ledger that records every step of the delivery, from the moment you place your order to when you receive your cheesy goodness. Each step – order confirmation, pizza preparation, driver dispatch, delivery – is recorded as a "block" of information.  These blocks are linked together chronologically and cryptographically secured, meaning no single entity can alter past records without everyone else noticing.

Every participant involved – you, the pizzeria, and the delivery driver – gets a copy of this ledger.  This transparency ensures everyone agrees on the pizza's journey. If the driver claims they delivered the pizza when they didn't, the immutable record on the blockchain proves otherwise.  This decentralized nature is key; no single person or company controls the information.

Because the blockchain is shared and every change is verified, it's incredibly secure and trustworthy.  It's far more reliable than a single, 

# simple Calculator + Chat Bot

In [ ]:
import requests
import google.generativeai as genai

genai.configure(api_key="****************")

memory = []
last_city = ""

def check_what_user_wants(text):
    if "weather" in text:
        return "weather"
    if "fashion" in text or "trend" in text:
        return "fashion"
    if "calculate" in text or any(num in text for num in "0123456789"):
        return "math"
    return "chat"

def get_weather(city):
    coords = {
        "mumbai": (19.0760, 72.8777),
        "delhi": (28.6139, 77.2090),
        "bangalore": (12.9716, 77.5946),
        "new york": (40.7128, -74.0060),
        "london": (51.5074, -0.1278),
        "paris": (48.8566, 2.3522),
        "tokyo": (35.6762, 139.6503),
        "sydney": (-33.8688, 151.2093)
    }
    
    lat, lon = coords[city.lower()]
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,weather_code&timezone=auto"
    r = requests.get(url)
    data = r.json()
    temp = data["current"]["temperature_2m"]
    return f"Weather in {city}: {temp}°C"

def get_fashion(city):
    fashion_stuff = {
        "mumbai": "Light cotton and ethnic wear",
        "delhi": "Layered clothing and scarves",
        "bangalore": "Smart casual and tech stuff",
        "new york": "Oversized blazers and accessories",
        "paris": "Vintage scarves and coats",
        "london": "Trench coats and boots",
        "tokyo": "Streetwear and colorful accessories",
        "sydney": "Beach wear and sun protection"
    }
    return f"Fashion in {city}: {fashion_stuff[city.lower()]}"

def do_math(text):
    answer = eval(text)
    return f"{text} = {answer}"

def find_city(text):
    cities = ["mumbai", "delhi", "bangalore", "new york", "london", "paris", "tokyo", "sydney"]
    for city in cities:
        if city in text.lower():
            return city
    return last_city

def answer_user(user_question):
    global last_city
    
    what_they_want = check_what_user_wants(user_question)
    
    if what_they_want == "weather":
        city = find_city(user_question)
        last_city = city
        result = get_weather(city)
        memory.append(user_question + " -> " + result)
        return result
    
    elif what_they_want == "fashion":
        city = find_city(user_question)
        last_city = city
        result = get_fashion(city)
        memory.append(user_question + " -> " + result)
        return result
    
    elif what_they_want == "math":
        result = do_math(user_question)
        memory.append(user_question + " -> " + result)
        return result
    
    else:
        if "hello" in user_question.lower():
            result = "Hello! I can help with weather, fashion, and math!"
        elif "how are you" in user_question.lower():
            result = "I'm good! What do you need?"
        else:
            result = "I can help with weather, fashion trends, or calculations!"
        
        memory.append(user_question + " -> " + result)
        return result

print("My Weather and Fashion Bot")

questions = [
    "What's the weather in Tokyo?",
    "What about fashion trends there?",
    "15 * 8",
    "Hello! How are you?",
    "What's the weather there?",
]

for question in questions:
    print(f"\nQuestion: {question}")
    answer = answer_user(question)
    print(f"Answer: {answer}")


My Weather and Fashion Bot

Question: What's the weather in Tokyo?
Answer: Weather in tokyo: 16.6°C

Question: What about fashion trends there?
Answer: Fashion in tokyo: Streetwear and colorful accessories

Question: 15 * 8
Answer: 15 * 8 = 120

Question: Hello! How are you?
Answer: Hello! I can help with weather, fashion, and math!

Question: What's the weather there?
Answer: Weather in tokyo: 16.6°C


# Enhanced Calculator + Chat Bot and Testing 

In [ ]:
import google.generativeai as genai
import math
import re

GEMINI_API_KEY = "***********"
genai.configure(api_key=GEMINI_API_KEY)

def calculator(expression):
    try:
        cleaned = expression.replace('^', '**')
        result = eval(cleaned)
        return f"Result: {result}"
    except:
        return "Math error"

def chatbot(question):
    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content(question)
    return response.text

def check_if_math(text):
    if any(char in text for char in '+-*/^%'):
        return True
    if 'sqrt' in text:
        return True
    return False

def weathermind(user_input):
    if check_if_math(user_input):
        return calculator(user_input)
    else:
        return chatbot(user_input)

test_questions = [
    "Explain how thunderstorms form",
    "15**2 + 3 / 4",
    "Current humidity in New York", 
    "sqrt(144) + 5*3",
    "2^8"
]

print("WeatherMind 2.0")

for question in test_questions:
    print(f"Input: {question}")
    answer = weathermind(question)
    print(f"Output: {answer}")
    print("---")


WeatherMind 2.0
Input: Explain how thunderstorms form
Output: Thunderstorms form through a complex process involving atmospheric instability, moisture, and lift.  Here's a breakdown of the stages:

**1. Lifting Condensation Level (LCL):**  Warm, moist air near the surface needs to rise. This can happen in several ways:

* **Convection:**  The sun heats the ground, warming the air above it.  This warm air becomes less dense than the surrounding air and rises. This is the most common way thunderstorms form.
* **Frontal Lifting:**  When a warm air mass meets a cold air mass, the warmer, less dense air is forced to rise over the colder air.  This is common along weather fronts.
* **Orographic Lifting:**  Air is forced to rise as it encounters a mountain or hill.

As the air rises, it cools and expands.  Eventually, it reaches its dew point – the temperature at which the air becomes saturated and condensation begins. This point is the LCL.  Clouds start to form.

**2. Cumulus Stage:**  The 

In [ ]:
import google.generativeai as genai

genai.configure(api_key="**************")


In [90]:
pip install langchain-google-genai


In [91]:
!pip install requests


#  Weather & Fashion Tools and testing 

In [ ]:
import requests
import google.generativeai as genai

genai.configure(api_key="*****************")

def get_weather(city):
    cities = {
        "mumbai": (19.0760, 72.8777),
        "delhi": (28.6139, 77.2090),
        "bangalore": (12.9716, 77.5946),
        "new york": (40.7128, -74.0060),
        "london": (51.5074, -0.1278),
        "paris": (48.8566, 2.3522),
        "tokyo": (35.6762, 139.6503),
        "sydney": (-33.8688, 151.2093)
    }
    
    city = city.lower()
    if city in cities:
        lat, lon = cities[city]
    else:
        return f"Sorry, I don't know {city}"
    
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,weather_code&timezone=auto"
    response = requests.get(url)
    data = response.json()
    
    temp = data["current"]["temperature_2m"]
    weather_code = data["current"]["weather_code"]
    
    weather_types = {
        0: "Clear sky",
        1: "Mainly clear", 
        2: "Partly cloudy",
        3: "Overcast",
        45: "Foggy",
        48: "Depositing rime fog",
        51: "Light drizzle",
        61: "Slight rain",
        80: "Slight rain showers"
    }
    
    if weather_code in weather_types:
        weather_desc = weather_types[weather_code]
    else:
        weather_desc = "Variable conditions"
    
    return f"Weather in {city}: {temp}°C, {weather_desc}"

def get_fashion_trends(city):
    trends = {
        "mumbai": "Light cotton fabrics and ethnic fusion wear",
        "delhi": "Layered clothing and scarves for winter",
        "bangalore": "Smart casual and tech-friendly accessories",
        "new york": "Oversized blazers and minimalist accessories",
        "paris": "Vintage scarves and tailored coats", 
        "london": "Trench coats and ankle boots",
        "tokyo": "Layered streetwear and colorful accessories",
        "sydney": "Beach-casual and sun protection wear"
    }
    
    city = city.lower()
    if city in trends:
        return f"Popular fashion trends in {city}: {trends[city]}"
    else:
        return f"Fashion trends in {city}: Minimalist clothing and sustainable fashion"

def weathermind_agent(question):
    model = genai.GenerativeModel('gemini-1.5-flash')
    
    if "weather" in question.lower():
        for city in ["mumbai", "delhi", "bangalore", "new york", "london", "paris", "tokyo", "sydney"]:
            if city in question.lower():
                weather_info = get_weather(city)
                return weather_info
    
    if "fashion" in question.lower() or "trends" in question.lower():
        for city in ["mumbai", "delhi", "bangalore", "new york", "london", "paris", "tokyo", "sydney"]:
            if city in question.lower():
                fashion_info = get_fashion_trends(city)
                return fashion_info
    
    response = model.generate_content(question)
    return response.text

print("WeatherMind with Weather & Fashion Tools")


questions = [
    "What's the weather in Mumbai?",
    "What are the fashion trends in Tokyo?", 
    "What's the weather like in New York?",
    "What fashion trends are popular in Paris?",
    "Hello! How are you doing today?"
]

for i, question in enumerate(questions, 1):
    print(f"\nTest {i}: {question}")
    
    
    answer = weathermind_agent(question)
    print(f"Response: {answer}")


WeatherMind with Weather & Fashion Tools

Test 1: What's the weather in Mumbai?
Response: Weather in mumbai: 29.5°C, Overcast

Test 2: What are the fashion trends in Tokyo?
Response: Popular fashion trends in tokyo: Layered streetwear and colorful accessories

Test 3: What's the weather like in New York?
Response: Weather in new york: 18.7°C, Overcast

Test 4: What fashion trends are popular in Paris?
Response: Popular fashion trends in paris: Vintage scarves and tailored coats

Test 5: Hello! How are you doing today?
Response: I am doing well, thank you for asking!  As a large language model, I don't experience emotions or feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How can I help you today?



# Memory-Enabled Assistant and Testing  Memory System

In [ ]:
import requests
import google.generativeai as genai

genai.configure(api_key="****************")

memory = []
last_city = ""

def check_what_user_wants(text):
    if "weather" in text:
        return "weather"
    if "fashion" in text or "trend" in text:
        return "fashion"
    if "calculate" in text or any(num in text for num in "0123456789"):
        return "math"
    return "chat"

def get_weather(city):
    coords = {
        "mumbai": (19.0760, 72.8777),
        "delhi": (28.6139, 77.2090),
        "bangalore": (12.9716, 77.5946),
        "new york": (40.7128, -74.0060),
        "london": (51.5074, -0.1278),
        "paris": (48.8566, 2.3522),
        "tokyo": (35.6762, 139.6503),
        "sydney": (-33.8688, 151.2093)
    }
    
    lat, lon = coords[city.lower()]
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,weather_code&timezone=auto"
    r = requests.get(url)
    data = r.json()
    temp = data["current"]["temperature_2m"]
    return f"Weather in {city}: {temp}°C"

def get_fashion(city):
    fashion_stuff = {
        "mumbai": "Light cotton and ethnic wear",
        "delhi": "Layered clothing and scarves",
        "bangalore": "Smart casual and tech stuff",
        "new york": "Oversized blazers and accessories",
        "paris": "Vintage scarves and coats",
        "london": "Trench coats and boots",
        "tokyo": "Streetwear and colorful accessories",
        "sydney": "Beach wear and sun protection"
    }
    return f"Fashion in {city}: {fashion_stuff[city.lower()]}"

def do_math(text):
    math_expression = ""
    for char in text:
        if char in "0123456789+-*/.() ":
            math_expression += char
    answer = eval(math_expression)
    return f"{math_expression} = {answer}"

def find_city(text):
    cities = ["mumbai", "delhi", "bangalore", "new york", "london", "paris", "tokyo", "sydney"]
    for city in cities:
        if city in text.lower():
            return city
    return last_city

def answer_user(user_question):
    global last_city
    
    what_they_want = check_what_user_wants(user_question)
    
    if what_they_want == "weather":
        city = find_city(user_question)
        last_city = city
        result = get_weather(city)
        memory.append(user_question + " -> " + result)
        return result
    
    elif what_they_want == "fashion":
        city = find_city(user_question)
        last_city = city
        result = get_fashion(city)
        memory.append(user_question + " -> " + result)
        return result
    
    elif what_they_want == "math":
        result = do_math(user_question)
        memory.append(user_question + " -> " + result)
        return result
    
    else:
        if "hello" in user_question.lower():
            result = "Hello! I can help with weather, fashion, and math!"
        elif "how are you" in user_question.lower():
            result = "I'm good! What do you need?"
        else:
            result = "I can help with weather, fashion trends, or calculations!"
        
        memory.append(user_question + " -> " + result)
        return result

print("My Weather and Fashion Bot")

questions = [
    "What's the weather in Tokyo?",
    "What about fashion trends there?",
    "Calculate 15 * 8",
    "Hello! How are you?",
    "What's the weather there?",
]

for question in questions:
    print(f"\nQuestion: {question}")
    answer = answer_user(question)
    print(f"Answer: {answer}")


My Weather and Fashion Bot

Question: What's the weather in Tokyo?
Answer: Weather in tokyo: 16.6°C

Question: What about fashion trends there?
Answer: Fashion in tokyo: Streetwear and colorful accessories

Question: Calculate 15 * 8
Answer:  15 * 8 = 120

Question: Hello! How are you?
Answer: Hello! I can help with weather, fashion, and math!

Question: What's the weather there?
Answer: Weather in tokyo: 16.6°C


In [ ]:
import requests
import google.generativeai as genai

genai.configure(api_key="*******************")

memory = []
last_city = ""

def check_what_user_wants(text):
    if "weather" in text:
        return "weather"
    if "fashion" in text or "trend" in text:
        return "fashion"
    if "calculate" in text or any(num in text for num in "0123456789"):
        return "math"
    return "chat"

def get_weather(city):
    coords = {
        "mumbai": (19.0760, 72.8777),
        "delhi": (28.6139, 77.2090),
        "bangalore": (12.9716, 77.5946),
        "new york": (40.7128, -74.0060),
        "london": (51.5074, -0.1278),
        "paris": (48.8566, 2.3522),
        "tokyo": (35.6762, 139.6503),
        "sydney": (-33.8688, 151.2093)
    }
    
    lat, lon = coords[city.lower()]
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,weather_code&timezone=auto"
    r = requests.get(url)
    data = r.json()
    temp = data["current"]["temperature_2m"]
    return f"Weather in {city}: {temp}°C"

def get_fashion(city):
    fashion_stuff = {
        "mumbai": "Light cotton and ethnic wear",
        "delhi": "Layered clothing and scarves",
        "bangalore": "Smart casual and tech stuff",
        "new york": "Oversized blazers and accessories",
        "paris": "Vintage scarves and coats",
        "london": "Trench coats and boots",
        "tokyo": "Streetwear and colorful accessories",
        "sydney": "Beach wear and sun protection"
    }
    return f"Fashion in {city}: {fashion_stuff[city.lower()]}"

def do_math(text):
    answer = eval(text)
    return f"{text} = {answer}"

def find_city(text):
    cities = ["mumbai", "delhi", "bangalore", "new york", "london", "paris", "tokyo", "sydney"]
    for city in cities:
        if city in text.lower():
            return city
    return last_city

def answer_user(user_question):
    global last_city
    
    what_they_want = check_what_user_wants(user_question)
    
    if what_they_want == "weather":
        city = find_city(user_question)
        last_city = city
        result = get_weather(city)
        memory.append(user_question + " -> " + result)
        return result
    
    elif what_they_want == "fashion":
        city = find_city(user_question)
        last_city = city
        result = get_fashion(city)
        memory.append(user_question + " -> " + result)
        return result
    
    elif what_they_want == "math":
        result = do_math(user_question)
        memory.append(user_question + " -> " + result)
        return result
    
    else:
        if "hello" in user_question.lower():
            result = "Hello! I can help with weather, fashion, and math!"
        elif "how are you" in user_question.lower():
            result = "I'm good! What do you need?"
        else:
            result = "I can help with weather, fashion trends, or calculations!"
        
        memory.append(user_question + " -> " + result)
        return result

print("My Weather and Fashion Bot")

questions = [
    "What's the weather in Tokyo?",
    "What about fashion trends there?",
    "15 * 8",
    "Hello! How are you?",
    "What's the weather there?",
]

for question in questions:
    print(f"\nQuestion: {question}")
    answer = answer_user(question)
    print(f"Answer: {answer}")


My Weather and Fashion Bot

Question: What's the weather in Tokyo?
Answer: Weather in tokyo: 16.6°C

Question: What about fashion trends there?
Answer: Fashion in tokyo: Streetwear and colorful accessories

Question: 15 * 8
Answer: 15 * 8 = 120

Question: Hello! How are you?
Answer: Hello! I can help with weather, fashion, and math!

Question: What's the weather there?
Answer: Weather in tokyo: 16.6°C


## running different custom tests


In [95]:
my_query = "What's the weather in Mumbai?"

print(f"\nYour Custom Test: {my_query}")

response = weathermind_multiagent(my_query)

print(f"Response: {response}")



Your Custom Test: What's the weather in Mumbai?
Response: Weather in mumbai: 29.5°C


In [96]:


# Test 1: Weather
print("test 1: weather")
result1 = answer_user("whats the weather in mumbai?")
print(f"question: whats the weather in mumbai?")
print(f"answer: {result1}")
print()

# Test 2: Fashion  
print("test 2: fashion")
result2 = answer_user("fashion trends in tokyo?")
print(f"question: fashion trends in tokyo?")
print(f"answer: {result2}")
print()

# Test 3: Math
print("test 3: math")
result3 = answer_user("10 + 15")
print(f"question: 10 + 15")
print(f"answer: {result3}")
print()

# Test 4: Chat
print("test 4: chat")
result4 = answer_user("hello")
print(f"question: hello")
print(f"answer: {result4}")
print()

# Test 5: Memory (remembers last city)
print("test 5: memory")
answer_user("weather in paris?")  # Set paris as last city
result5 = answer_user("fashion trends there?")  # Should use paris
print(f"question: fashion trends there?")
print(f"answer: {result5}")
print()

print("all tests completed!")


test 1: weather
question: whats the weather in mumbai?
answer: Weather in mumbai: 29.5°C

test 2: fashion
question: fashion trends in tokyo?
answer: Fashion in tokyo: Streetwear and colorful accessories

test 3: math
question: 10 + 15
answer: 10 + 15 = 25

test 4: chat
question: hello
answer: Hello! I can help with weather, fashion, and math!

test 5: memory
question: fashion trends there?
answer: Fashion in paris: Vintage scarves and coats

all tests completed!
